In [1]:

import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql
import os
from sqlite3 import Error

In [2]:
# Try different encodings
try:
    e_com = pd.read_csv('Sample-Superstore.csv', encoding='utf-8')
except UnicodeDecodeError:
    # Try alternative encodings
    e_com = pd.read_csv('Sample-Superstore.csv', encoding='latin-1')


In [3]:
E_Commerce = e_com

In [4]:
E_Commerce.shape

(9994, 21)

In [5]:
E_Commerce.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [6]:
E_Commerce['Ship Mode']

0         Second Class
1         Second Class
2         Second Class
3       Standard Class
4       Standard Class
             ...      
9989      Second Class
9990    Standard Class
9991    Standard Class
9992    Standard Class
9993      Second Class
Name: Ship Mode, Length: 9994, dtype: object

In [7]:
#change the column names to lowercase and replace spaces with underscores
E_Commerce.columns = E_Commerce.columns.str.lower().str.replace(' ', '_')

In [8]:
#find missing null values
E_Commerce.isnull().sum()

#drop missing values
E_Commerce = E_Commerce.dropna()

In [9]:
# Convert started_at and ended_at to datetime for memory efficiency
E_Commerce['order_date'] = pd.to_datetime(E_Commerce['order_date'])
E_Commerce['ship_date'] = pd.to_datetime(E_Commerce['ship_date'])

# FUTURE ENGINEERING 

## ADDING NEW COLUMNS 

In [10]:



# Extracting year, month From  ship date for shipping
E_Commerce['ship_year'] = E_Commerce['ship_date'].dt.year
E_Commerce['ship_month'] = E_Commerce['ship_date'].dt.month_name()
E_Commerce['ship_month_no'] = E_Commerce['ship_date'].dt.month

In [11]:
# Adding new column for Shipping Season 
E_Commerce['ship_season'] = E_Commerce['ship_month_no'].apply(lambda value: 'winter'if value in (12, 1, 2)
                                                                 else 'Spring'if value in (3, 4, 5) 
                                                                 else 'Summer'if value in (6, 7, 8) 
                                                                 else 'Fall' )

In [12]:
high_sales_threshold = 15000
medium_sales_threshold = 8000

# Categorize sales using pd.cut()
E_Commerce['sales_category'] = pd.cut(
    E_Commerce['sales'],
    bins=[-float('inf'), medium_sales_threshold, high_sales_threshold, float('inf')],
    labels=['Low', 'Medium', 'High']
)



In [13]:


# Add new 'ship_id' column
E_Commerce['ship_id'] = None  # Or assign default values
E_Commerce['region_id'] = None
E_Commerce['profit_id'] = None


In [14]:
#Adding new column for Total Revenue
E_Commerce['total_revenue'] = E_Commerce['sales'] * E_Commerce['quantity']

In [15]:
#Adding new column for Profit and Loss
E_Commerce['status'] = E_Commerce['profit'].apply(lambda x: 'Profit' if x > 0 else 'Loss')

In [16]:
# Check for duplicates in OrderID and ProductID
duplicates = E_Commerce[E_Commerce.duplicated(subset=["order_id", "product_id"], keep=False)]

# Display duplicates
print(duplicates)

      row_id        order_id order_date  ship_date       ship_mode  \
350      351  CA-2016-129714 2016-09-01 2016-09-03     First Class   
352      353  CA-2016-129714 2016-09-01 2016-09-03     First Class   
430      431  US-2016-123750 2016-04-15 2016-04-21  Standard Class   
431      432  US-2016-123750 2016-04-15 2016-04-21  Standard Class   
1300    1301  CA-2016-137043 2016-12-23 2016-12-25    Second Class   
1301    1302  CA-2016-137043 2016-12-23 2016-12-25    Second Class   
3183    3184  CA-2017-152912 2017-11-09 2017-11-12    Second Class   
3184    3185  CA-2017-152912 2017-11-09 2017-11-12    Second Class   
3405    3406  US-2014-150119 2014-04-23 2014-04-27  Standard Class   
3406    3407  US-2014-150119 2014-04-23 2014-04-27  Standard Class   
6498    6499  CA-2015-103135 2015-07-24 2015-07-28  Standard Class   
6500    6501  CA-2015-103135 2015-07-24 2015-07-28  Standard Class   
7881    7882  CA-2017-118017 2017-12-03 2017-12-06    Second Class   
7882    7883  CA-201

In [17]:
# Rename columns in the DataFrame
E_Commerce = E_Commerce.rename(columns={
    'segment': 'segments',
    'sub-category': 'sub_category',
    'sales': 'sale',
    'ship_month': 'ship_month_name'
})

In [21]:
# Rename columns in the DataFrame
E_Commerce = E_Commerce.rename(columns={
    'sub-category': 'sub_category',  # Rename 'sub-category' to 'sub_category'
    'segments': 'segment'            # Rename 'segment' to 'segments'
})

# Login To SQL

In [22]:
import sqlite3

# Create SQLite connection
conn = sqlite3.connect("superstore.db")
cursor = conn.cursor()

# Create Tables
cursor.executescript('''
-- Customers Table
CREATE TABLE IF NOT EXISTS Customers (
    customer_id VARCHAR(20) PRIMARY KEY,
    customer_name VARCHAR(100)
);

-- Orders Table
CREATE TABLE IF NOT EXISTS Orders (
    order_id VARCHAR(20) PRIMARY KEY,
    order_date DATE,
    ship_mode VARCHAR(50),
    customer_id VARCHAR(20),
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

-- Products Table
CREATE TABLE IF NOT EXISTS Products (
    product_id VARCHAR(20) PRIMARY KEY,
    product_name VARCHAR(255),
    category VARCHAR(50),
    sub_category VARCHAR(50),
    segment VARCHAR(50)
);

-- Shipment Table
CREATE TABLE IF NOT EXISTS Shipment (
    ship_id INTEGER PRIMARY KEY AUTOINCREMENT,
    ship_date DATE,
    ship_year INT,
    ship_month_no INT,
    ship_month_name VARCHAR(20),
    ship_season VARCHAR(20),
    order_id VARCHAR(20),
    FOREIGN KEY (order_id) REFERENCES Orders(order_id)
);

-- Regions Table
CREATE TABLE IF NOT EXISTS Regions (
    region_id INTEGER PRIMARY KEY AUTOINCREMENT,
    country VARCHAR(100),
    region VARCHAR(50),
    state VARCHAR(50),
    city VARCHAR(100),
    postal_code INT
);

-- Profits Table
CREATE TABLE IF NOT EXISTS Profits (
    profit_id INTEGER PRIMARY KEY AUTOINCREMENT,
    profit DECIMAL(10, 2),
    sale DECIMAL(10, 2),
    total_revenue DECIMAL(10, 2),
    status VARCHAR(100),
    sales_category VARCHAR(100),
    discount DECIMAL(5, 2),
    quantity INT,
    product_id VARCHAR(20),
    FOREIGN KEY (product_id) REFERENCES Products(product_id)
);

''')

# Commit and close connection
conn.commit()
conn.close()

In [23]:
import pandas as pd
import sqlite3

def populate_database_tables(conn, E_Commerce):
    """
    Step 2: Populates the database tables with data from the enriched DataFrame.
    
    Args:
        conn (sqlite3.Connection): The database connection
        E_Commerce (pandas.DataFrame): Enriched DataFrame with the column names as shown
        
    Returns:
        sqlite3.Connection: The database connection
    """
    cursor = conn.cursor()
    
    try:
        # Customers
        customers_df = E_Commerce[['customer_id', 'customer_name']].drop_duplicates()
        for _, row in customers_df.iterrows():
            try:
                cursor.execute(
                    "INSERT OR IGNORE INTO Customers (customer_id, customer_name) VALUES (?, ?)",
                    (row['customer_id'], row['customer_name'])
                )
            except sqlite3.Error as e:
                print(f"Error inserting into Customers: {e}")

        # Orders
        orders_df = E_Commerce[['order_id', 'order_date', 'ship_mode', 'customer_id']].drop_duplicates()
        for _, row in orders_df.iterrows():
            try:
                order_date_str = row['order_date'].strftime('%Y-%m-%d') if pd.notna(row['order_date']) else None
                cursor.execute(
                    "INSERT OR IGNORE INTO Orders (order_id, order_date, ship_mode, customer_id) VALUES (?, ?, ?, ?)",
                    (row['order_id'], order_date_str, row['ship_mode'], row['customer_id'])
                )
            except sqlite3.Error as e:
                print(f"Error inserting into Orders: {e}")

        # Products
        products_df = E_Commerce[['product_id', 'product_name', 'category', 'sub_category', 'segment']].drop_duplicates()
        for _, row in products_df.iterrows():
            try:
                cursor.execute(
                    "INSERT OR IGNORE INTO Products (product_id, product_name, category, sub_category, segment) VALUES (?, ?, ?, ?, ?)",
                    (row['product_id'], row['product_name'], row['category'], row['sub_category'], row['segment'])
                )
            except sqlite3.Error as e:
                print(f"Error inserting into Products: {e}")

        # Regions
        regions_df = E_Commerce[['region_id', 'country', 'region', 'state', 'city', 'postal_code']].drop_duplicates()
        for _, row in regions_df.iterrows():
            try:
                cursor.execute(
                    "INSERT OR IGNORE INTO Regions (region_id, country, region, state, city, postal_code) VALUES (?, ?, ?, ?, ?, ?)",
                    (row['region_id'], row['country'], row['region'], row['state'], row['city'], row['postal_code'])
                )
            except sqlite3.Error as e:
                print(f"Error inserting into Regions: {e}")

        # Profits
        profits_df = E_Commerce[['profit_id', 'profit', 'sale', 'total_revenue', 'status', 'sales_category', 'discount', 'quantity', 'product_id']].drop_duplicates()
        for _, row in profits_df.iterrows():
            try:
                cursor.execute(
                    "INSERT OR IGNORE INTO Profits (profit_id, profit, sale, total_revenue, status, sales_category, discount, quantity, product_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
                    (row['profit_id'], row['profit'], row['sale'], row['total_revenue'], row['status'], row['sales_category'], row['discount'], row['quantity'], row['product_id'])
                )
            except sqlite3.Error as e:
                print(f"Error inserting into Profits: {e}")

        # Shipment
        shipment_df = E_Commerce[['ship_id', 'ship_date', 'ship_year', 'ship_month_no', 'ship_month_name', 'ship_season', 'order_id']].drop_duplicates()
        for _, row in shipment_df.iterrows():
            try:
                ship_date_str = row['ship_date'].strftime('%Y-%m-%d') if pd.notna(row['ship_date']) else None
                cursor.execute(
                    "INSERT OR IGNORE INTO Shipment (ship_id, ship_date, ship_year, ship_month_no, ship_month_name, ship_season, order_id) VALUES (?, ?, ?, ?, ?, ?, ?)",
                    (row['ship_id'], ship_date_str, row['ship_year'], row['ship_month_no'], row['ship_month_name'], row['ship_season'], row['order_id'])
                )
            except sqlite3.Error as e:
                print(f"Error inserting into Shipment: {e}")

        # Commit changes
        conn.commit()
        print("Tables populated successfully")
    
    except Exception as e:
        print(f"An error occurred: {e}")
    
    return conn

In [24]:
# Check column names
print(E_Commerce.columns)

# Inspect the first few rows
print(E_Commerce.head())

# Check for missing values
print(E_Commerce.isnull().sum())

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'customer_name', 'segment', 'country', 'city', 'state',
       'postal_code', 'region', 'product_id', 'category', 'sub_category',
       'product_name', 'sale', 'quantity', 'discount', 'profit', 'ship_year',
       'ship_month_name', 'ship_month_no', 'ship_season', 'sales_category',
       'ship_id', 'region_id', 'profit_id', 'total_revenue', 'status'],
      dtype='object')
   row_id        order_id order_date  ship_date       ship_mode customer_id  \
0       1  CA-2016-152156 2016-11-08 2016-11-11    Second Class    CG-12520   
1       2  CA-2016-152156 2016-11-08 2016-11-11    Second Class    CG-12520   
2       3  CA-2016-138688 2016-06-12 2016-06-16    Second Class    DV-13045   
3       4  US-2015-108966 2015-10-11 2015-10-18  Standard Class    SO-20335   
4       5  US-2015-108966 2015-10-11 2015-10-18  Standard Class    SO-20335   

     customer_name    segment        country            

In [25]:
# Open the database connection
conn = sqlite3.connect("superstore.db")

# Call the function to populate tables
populate_database_tables(conn, E_Commerce)

# Close the connection after the function completes
conn.close()

Tables populated successfully


In [26]:
conn = sqlite3.connect("superstore.db")
cursor = conn.cursor()

# Query Customers Table
print("Customers Table:")
cursor.execute("SELECT * FROM Customers")
print(cursor.fetchall())

# Query Orders Table
print("\nOrders Table:")
cursor.execute("SELECT * FROM Orders")
print(cursor.fetchall())

# Query Products Table
print("\nProducts Table:")
cursor.execute("SELECT * FROM Products")
print(cursor.fetchall())

# Query Regions Table
print("\nRegions Table:")
cursor.execute("SELECT * FROM Regions")
print(cursor.fetchall())

# Query Profits Table
print("\nProfits Table:")
cursor.execute("SELECT * FROM Profits")
print(cursor.fetchall())

# Query Shipment Table
print("\nShipment Table:")
cursor.execute("SELECT * FROM Shipment")
print(cursor.fetchall())

conn.close()

Customers Table:
[('CG-12520', 'Claire Gute'), ('DV-13045', 'Darrin Van Huff'), ('SO-20335', "Sean O'Donnell"), ('BH-11710', 'Brosina Hoffman'), ('AA-10480', 'Andrew Allen'), ('IM-15070', 'Irene Maddox'), ('HP-14815', 'Harold Pawlan'), ('PK-19075', 'Pete Kriz'), ('AG-10270', 'Alejandro Grove'), ('ZD-21925', 'Zuschuss Donatelli'), ('KB-16585', 'Ken Black'), ('SF-20065', 'Sandra Flanagan'), ('EB-13870', 'Emily Burns'), ('EH-13945', 'Eric Hoffmann'), ('TB-21520', 'Tracy Blumstein'), ('MA-17560', 'Matt Abelman'), ('GH-14485', 'Gene Hale'), ('SN-20710', 'Steve Nguyen'), ('LC-16930', 'Linda Cazamias'), ('RA-19885', 'Ruben Ausman'), ('ES-14080', 'Erin Smith'), ('ON-18715', 'Odella Nelson'), ('PO-18865', "Patrick O'Donnell"), ('LH-16900', 'Lena Hernandez'), ('DP-13000', 'Darren Powers'), ('JM-15265', 'Janet Molinari'), ('TB-21055', 'Ted Butterfield'), ('KM-16720', 'Kunst Miller'), ('PS-18970', 'Paul Stevenson'), ('BS-11590', 'Brendan Sweed'), ('KD-16270', 'Karen Daniels'), ('HM-14980', 'Henry 

In [27]:
E_Commerce.to_csv('E_Commerce.csv', index=False)